In [1]:
# Import the necessary libraries if you are starting a new notebook.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils.index import *

In [2]:
url = "../03-cleaning-categorical-data/files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv"
df = pd.read_csv(url)
df.columns = standardize_columns_names(df.columns)
df.set_index("customer", inplace=True)

In [3]:
df.head(5).transpose()

customer,BU79786,QZ44356,AI49188,WW63253,HB64268
state,Washington,Arizona,Nevada,California,Washington
customer_lifetime_value,2763.519279,6979.535903,12887.43165,7645.861827,2813.692575
response,No,No,No,No,No
coverage,Basic,Extended,Premium,Basic,Basic
education,Bachelor,Bachelor,Bachelor,Bachelor,Bachelor
effective_to_date,2/24/11,1/31/11,2/19/11,1/20/11,2/3/11
employment_status,Employed,Unemployed,Employed,Unemployed,Employed
gender,F,F,F,M,M
income,56274,0,48767,0,43836
location_code,Suburban,Suburban,Suburban,Suburban,Rural


In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def apply_model(df: pd.DataFrame, target, model_list):
    X_train, X_test, y_train, y_test = split_data(df, target)
    
    X_train_scaled, X_test_scaled = scale_data(sets=[X_train, X_test], scaler=StandardScaler)

    X_train_scaled_and_encoded, X_test_scaled_and_encoded = encode_data(
        scaled_sets=[X_train_scaled, X_test_scaled],
        encoder=OneHotEncoder)
    
    trained_models = []
    for model in model_list:
        trained_models.append(model().fit(X_train_scaled_and_encoded, y_train))
    
    predictions_trained_models = []
    for trained_model in trained_models:
        predictions_trained_models.append(trained_model.predict(X_test_scaled_and_encoded))
    
    return predictions_trained_models, y_test



In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression      

df["response"] = df["response"].map({'Yes': 1, 'No': 0})

lst_y_predictions, y_test = apply_model(
    df,
    target="response",
    model_list=[LogisticRegression, KNeighborsClassifier])

/home/linuxbrew/.linuxbrew/Cellar/python@3.9/3.9.13_1/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/linuxbrew/.linuxbrew/Cellar/python@3.9/3.9.13_1/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/linuxbrew/.linuxbrew/Cellar/python@3.9/3.9.13_1/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names t

In [6]:
from sklearn.metrics import confusion_matrix

for y_pred in lst_y_predictions:
    display(confusion_matrix(y_test, y_pred))

array([[1939,   31],
       [ 253,   61]])

array([[1844,  126],
       [  31,  283]])